In [ ]:
##########################################
####Scenario 6############################
########cor=0.7,p=500,miss_prop=0.40#######
rand_vect <- function(n,mean, sd ) {
  vec <- rnorm(n,mean, sd)
  while (any(vec < 0)) {
    negs <- vec < 0
    vec[negs][i] <- vec[negs][i <- sample(sum(negs), 1)] + 1
    
  }
  vec
}
compute.sigma.star1<-function (no.bin, no.nor, prop.vec.bin = NULL, corr.vec = NULL,corr.mat = NULL) 
{
  d = no.bin + no.nor
  validation.corr(no.bin, no.nor, prop.vec.bin, corr.vec = corr.vec, 
                  corr.mat = corr.mat)
  if ((no.nor < 0) | (floor(no.nor) != no.nor)) {
    warning("Number of normal variables \nmust be a non-negative integer!\n")
  }
  if (is.null(corr.mat)) {
    corr.mat = lower.tri.to.corr.mat(corr.vec, d)
  }
  sigma = corr.mat
  p = prop.vec.bin
  q = 1 - p
  if (no.bin != 0) {
    sigmaBB = diag(no.bin)
    for (i in 1:no.bin) {
      for (j in 1:no.bin) {
        if (i != j) 
          sigmaBB[i, j] = phi2tetra(sigma[i, j], c(p[i], 
                                                   p[j]))
      }
    }
  }
  if (no.bin > 0 & no.nor > 0) {
    sigmaBN = sigma
    for (i in (no.bin + 1):d) {
      for (j in 1:no.bin) {
        sigmaBN[i, j] = sigmaBN[i, j]/(dnorm(qnorm(p[j]))/sqrt(p[j] * 
                                                                 q[j]))
      }
    }
    sigmaBN = sigmaBN[(no.bin + 1):d, 1:no.bin]
    sigma_star = sigma
    sigma_star[1:no.bin, 1:no.bin] = sigmaBB
    sigma_star[(no.bin + 1):d, 1:no.bin] = sigmaBN
    sigma_star[1:no.bin, (no.bin + 1):d] = t(sigmaBN)
  }
  if (no.bin > 0 & no.nor == 0) {
    sigma_star = sigmaBB
  }
  if (no.bin == 0 & no.nor > 0) {
    sigma_star = sigma
  }
  PD = TRUE
  temp = NULL
  eigenv = eigen(sigma_star)$value
  if (is.positive.definite(sigma_star) == FALSE) {
    temp = sigma_star
    cat("sigma_star before using nearPD is\n")
    print(temp)
    sigma_star = as.matrix(nearPD(sigma_star, corr = TRUE, 
                                  keepDiag = TRUE,maxit = 10000)$mat)
    sigma_star = (sigma_star + t(sigma_star))/2
    warning("sigma_star is not positive definite.\nAlgorithm will be using the nearest positive definite matrix which is\n", 
            immediate. = TRUE)
    print(sigma_star)
    PD = FALSE
  }
  return(list(sigma_star = sigma_star, nonPD = temp, PD = PD, 
              eigenv = eigenv))
}
M<-1000
perf_measure_train<-matrix(nrow=M,ncol=4)
perf_measure_test<-matrix(nrow=M,ncol=4)
perf_measure_amputed_mcar_train<-matrix(nrow=M,ncol=4)
perf_measure_amputed_mcar_test<-matrix(nrow=M,ncol=4)
perf_measure_amputed_mar_train<-matrix(nrow=M,ncol=4)
perf_measure_amputed_mar_test<-matrix(nrow=M,ncol=4)
perf_measure_amputed_mnar_train<-matrix(nrow=M,ncol=4)
perf_measure_amputed_mnar_test<-matrix(nrow=M,ncol=4)
perf_measure_imputed_mcar_train<-matrix(nrow=M,ncol=4)
perf_measure_imputed_mcar_test<-matrix(nrow=M,ncol=4)
perf_measure_imputed_mar_train<-matrix(nrow=M,ncol=4)
perf_measure_imputed_mar_test<-matrix(nrow=M,ncol=4)
perf_measure_imputed_mnar_train<-matrix(nrow=M,ncol=4)
perf_measure_imputed_mnar_test<-matrix(nrow=M,ncol=4)

for(i in 1:M){
  library(BinNor)
  numrows=10000
  numbin=1
  numnor=500#######
  no.rows=numrows
  proportion=0.5
  no.bin=numbin; no.nor=numnor
  mean_meanvec<-5
  sd_meanvec<-2
  mean_variancevec<-1
  sd_variancevec<-0.2
  means<-rnorm(mean=mean_meanvec,sd=sd_meanvec,n=numnor)
  vars<-rand_vect(mean=mean_variancevec,sd=sd_variancevec,n=numnor)
  mean.vec.nor=means; var.nor=vars
  prop.vec.bin=proportion
  upperdiag<-choose((numnor+1),2)
  cor=0.7############
  cor_sd<-0.0001
  corr.vec<-rnorm(mean=cor,sd=cor_sd,n=upperdiag)
  cmat = lower.tri.to.corr.mat(corr.vec,(numnor+1))
  sigma.star=compute.sigma.star1(no.bin=numbin, no.nor=numnor, prop.vec.bin=proportion,
                                corr.mat=cmat)
  mydata=jointly.generate.binary.normal(no.rows,no.bin,no.nor,prop.vec.bin,
                                        mean.vec.nor,var.nor, sigma_star=sigma.star$sigma_star,
                                        continue.with.warning=TRUE)
  mydata<-data.frame(mydata)
  
  library(mice)
  library(caret)
  miss_prop=0.40
  data.m<-ampute(mydata,prop = miss_prop,mech="MAR")
  data_mar<-data.m$amp
  data.m1<-ampute(mydata,prop = miss_prop,mech="MCAR")
  data_mcar<-data.m1$amp
  data.m2<-ampute(mydata,prop = miss_prop,mech="MNAR")
  data_mnar<-data.m2$amp
  library(Amelia)
  #missmap(data_mar)
  #missmap(data_mcar)
  #missmap(data_mnar)
  imputed_mar<-amelia(x=data_mar,m=1,noms="X1")
  imputed_mar_data<-data.frame(imputed_mar$imputations[[1]])
  imputed_mcar<-amelia(x=data_mcar,m=1,noms="X1")
  imputed_mcar_data<-data.frame(imputed_mcar$imputations[[1]])
  imputed_mnar<-amelia(x=data_mnar,m=1,noms="X1")
  imputed_mnar_data<-data.frame(imputed_mnar$imputations[[1]])
  ID<-c(1:10000)
  train_ID<-sample(ID,size=8000)
  test_ID<-ID[-train_ID]
  #######Full Data set test and train####
  full_train<-mydata[train_ID,]
  full_test<-mydata[test_ID,]
  ####Amputed Data Sets
  amputed_mar_data_train<-data_mar[train_ID,]
  amputed_mcar_data_train<-data_mcar[train_ID,]
  amputed_mnar_data_train<-data_mnar[train_ID,]
  amputed_mar_data_test<-data_mar[test_ID,]
  amputed_mcar_data_test<-data_mcar[test_ID,]
  amputed_mnar_data_test<-data_mcar[test_ID,]
  ######Imputed Data Sets
  imputed_mar_data_train<-imputed_mar_data[train_ID,]
  imputed_mcar_data_train<-imputed_mcar_data[train_ID,]
  imputed_mnar_data_train<-imputed_mnar_data[train_ID,]
  imputed_mar_data_test<-imputed_mar_data[test_ID,]
  imputed_mcar_data_test<-imputed_mcar_data[test_ID,]
  imputed_mnar_data_test<-imputed_mnar_data[test_ID,]
  ####Deep Learning
  library(h2o)
  h2o.init()
  ####H2o enviroment data sets for full data
  full_data_train_h2o<-as.h2o(full_train)
  full_data_test_h2o<-as.h2o(full_test)
  full_data_train_h2o[,1]<-as.factor(full_data_train_h2o[,1])
  full_data_test_h2o[,1]<-as.factor(full_data_test_h2o[,1])
  #######H2o enviroment data sets for amputed data
  amputed_mar_data_train_h2o<-as.h2o(amputed_mar_data_train)
  amputed_mcar_data_train_h2o<-as.h2o(amputed_mcar_data_train)
  amputed_mnar_data_train_h2o<-as.h2o(amputed_mnar_data_train)
  amputed_mar_data_test_h2o<-as.h2o(amputed_mar_data_test)
  amputed_mcar_data_test_h2o<-as.h2o(amputed_mcar_data_test)
  amputed_mnar_data_test_h2o<-as.h2o(amputed_mnar_data_test)
  amputed_mar_data_train_h2o[,1]<-as.factor(amputed_mar_data_train_h2o[,1])
  amputed_mcar_data_train_h2o[,1]<-as.factor(amputed_mcar_data_train_h2o[,1])
  amputed_mnar_data_train_h2o[,1]<-as.factor(amputed_mnar_data_train_h2o[,1])
  amputed_mar_data_test_h2o[,1]<-as.factor(amputed_mar_data_test_h2o[,1])
  amputed_mcar_data_test_h2o[,1]<-as.factor(amputed_mcar_data_test_h2o[,1])
  amputed_mnar_data_test_h2o[,1]<-as.factor(amputed_mnar_data_test_h2o[,1])
  #######H2o enviroment data sets for Imputed data
  imputed_mar_data_train_h2o<-as.h2o(imputed_mar_data_train)
  imputed_mcar_data_train_h2o<-as.h2o(imputed_mcar_data_train)
  imputed_mnar_data_train_h2o<-as.h2o(imputed_mnar_data_train)
  imputed_mar_data_test_h2o<-as.h2o(imputed_mar_data_test)
  imputed_mcar_data_test_h2o<-as.h2o(imputed_mcar_data_test)
  imputed_mnar_data_test_h2o<-as.h2o(imputed_mnar_data_test)
  imputed_mar_data_train_h2o[,1]<-as.factor(imputed_mar_data_train_h2o[,1])
  imputed_mcar_data_train_h2o[,1]<-as.factor(imputed_mcar_data_train_h2o[,1])
  imputed_mnar_data_train_h2o[,1]<-as.factor(imputed_mnar_data_train_h2o[,1])
  imputed_mar_data_test_h2o[,1]<-as.factor(imputed_mar_data_test_h2o[,1])
  imputed_mcar_data_test_h2o[,1]<-as.factor(imputed_mcar_data_test_h2o[,1])
  imputed_mnar_data_test_h2o[,1]<-as.factor(imputed_mnar_data_test_h2o[,1])
  ###########Hyperparameters
  hidden_opt<-list(c(rep(100,2)),c(rep(100,7)),c(rep(200,2)),c(rep(200,7)))
  hyper_param<-list(hidden=hidden_opt,
                    max_w2 = 10, l1=1e-5)
 
   
  folds=5
  ###Models and Performance for full data
  model_full <- h2o.grid("deeplearning",
                         x=2:(numnor+1), y=1,
                         nfolds=folds, 
                         hyper_params = hyper_param,
                         training_frame =full_data_train_h2o,
                         activation="Tanh")
  
  
  #**** Get the grid results, sorted by validation AUC
  model_full_grid <- h2o.getGrid(grid_id = model_full@grid_id,
                                 sort_by = "auc",
                                 decreasing = TRUE)
  #**** Grab the top Deep Learning model, chosen by validation AUC
  best_full <- h2o.getModel(model_full_grid@model_ids[[1]])
  
  #**** Now let's evaluate the model performance 
  perf_full_train<-h2o.performance(best_full)
  
  cm_full_train<-as.table(as.matrix(perf_full_train@metrics$cm$table[1:2,1:2]))
  measures_full_train<-confusionMatrix(cm_full_train,positive = "1")
  perf_measure_train[i,1:4]<-matrix(c(measures_full_train$overall[1],measures_full_train$byClass[1:2],
                                   perf_full_train@metrics$AUC),ncol=4)
  
  perf_full_test<-h2o.performance(best_full,newdata = full_data_test_h2o) #***test performance 
  
  cm_full_test<-as.table(as.matrix(perf_full_test@metrics$cm$table[1:2,1:2]))
  measures_full_test<-confusionMatrix(cm_full_test,positive = "1")
  perf_measure_test[i,1:4]<-matrix(c(measures_full_test$overall[1],measures_full_test$byClass[1:2],
                                  perf_full_test@metrics$AUC),ncol=4)
  
  ###########Models and Performance for amputed data
  model_amputed_mar<- h2o.grid("deeplearning",
                               x=2:(numnor+1), y=1,
                               nfolds=folds, 
                               hyper_params = hyper_param,
                               training_frame =amputed_mar_data_train_h2o,
                               activation="Tanh")
  
  #**** Get the grid results, sorted by validation AUC
  model_amputed_mar_grid <- h2o.getGrid(grid_id = model_amputed_mar@grid_id,
                                        sort_by = "auc",
                                        decreasing = TRUE)
  #**** Grab the top Deep Learning model, chosen by validation AUC
  best_amputed_mar <- h2o.getModel(model_amputed_mar_grid@model_ids[[1]])
  
  #**** Now let's evaluate the model performance 
  perf_amputed_train<-h2o.performance(best_amputed_mar)
  
  cm_amputed_mar_train<-as.table(as.matrix(perf_amputed_train@metrics$cm$table[1:2,1:2]))
  measures_full_train<-confusionMatrix(cm_amputed_mar_train,positive = "1")
  perf_measure_amputed_mar_train[i,1:4]<-matrix(c(measures_full_train$overall[1],measures_full_train$byClass[1:2],
                                               perf_amputed_train@metrics$AUC),ncol=4)
  
  perf_amputed_test<-h2o.performance(best_amputed_mar,newdata = amputed_mar_data_test_h2o) #***test performance
  
  cm_amputed_mar_test<-as.table(as.matrix(perf_amputed_test@metrics$cm$table[1:2,1:2]))
  measures_full_test<-confusionMatrix(cm_amputed_mar_test,positive = "1")
  perf_measure_amputed_mar_test[i,1:4]<-matrix(c(measures_full_test$overall[1],measures_full_test$byClass[1:2],
                                              perf_amputed_test@metrics$AUC),ncol=4)
  
  model_amputed_mcar<-h2o.grid("deeplearning",
                               x=2:(numnor+1), y=1,
                               nfolds=folds, 
                               hyper_params = hyper_param,
                               training_frame =amputed_mcar_data_train_h2o,
                               activation="Tanh")
  
  #**** Get the grid results, sorted by validation AUC
  model_amputed_mcar_grid <- h2o.getGrid(grid_id = model_amputed_mcar@grid_id,
                                         sort_by = "auc",
                                         decreasing = TRUE)
  
  #**** Grab the top Deep Learning model, chosen by validation AUC
  best_amputed_mcar <- h2o.getModel(model_amputed_mcar_grid@model_ids[[1]])
  
  #**** Now let's evaluate the model performance 
  perf_amputed_train<-h2o.performance(best_amputed_mcar)
  
  cm_amputed_mcar_train<-as.table(as.matrix(perf_amputed_train@metrics$cm$table[1:2,1:2]))
  measures_full_train<-confusionMatrix(cm_amputed_mcar_train,positive = "1")
  perf_measure_amputed_mcar_train[i,1:4]<-matrix(c(measures_full_train$overall[1],measures_full_train$byClass[1:2],
                                                perf_amputed_train@metrics$AUC),ncol=4)
  
  perf_amputed_test<-h2o.performance(best_amputed_mcar,newdata = amputed_mcar_data_test_h2o) #***test performance
  
  cm_amputed_mcar_test<-as.table(as.matrix(perf_amputed_test@metrics$cm$table[1:2,1:2]))
  measures_full_test<-confusionMatrix(cm_amputed_mcar_test,positive = "1")
  perf_measure_amputed_mcar_test[i,1:4]<-matrix(c(measures_full_test$overall[1],measures_full_test$byClass[1:2],
                                               perf_amputed_test@metrics$AUC),ncol=4)
  
  model_amputed_mnar<-h2o.grid("deeplearning",
                               x=2:(numnor+1), y=1,
                               nfolds=folds, 
                               hyper_params = hyper_param,
                               training_frame =amputed_mnar_data_train_h2o,
                               activation="Tanh")
  
  
  #**** Get the grid results, sorted by validation AUC
  model_amputed_mnar_grid <- h2o.getGrid(grid_id = model_amputed_mnar@grid_id,
                                         sort_by = "auc",
                                         decreasing = TRUE)
  
  #**** Grab the top Deep Learning model, chosen by validation AUC
  best_amputed_mnar <- h2o.getModel(model_amputed_mnar_grid@model_ids[[1]])
  
  #**** Now let's evaluate the model performance 
  perf_amputed_train<-h2o.performance(best_amputed_mnar)
  
  cm_amputed_mnar_train<-as.table(as.matrix(perf_amputed_train@metrics$cm$table[1:2,1:2]))
  measures_full_train<-confusionMatrix(cm_amputed_mnar_train,positive = "1")
  perf_measure_amputed_mnar_train[i,1:4]<-matrix(c(measures_full_train$overall[1],measures_full_train$byClass[1:2],
                                                perf_amputed_train@metrics$AUC),ncol=4)
  
  perf_amputed_test<-h2o.performance(best_amputed_mnar,newdata = amputed_mnar_data_test_h2o) #***test performance
  
  cm_amputed_mnar_test<-as.table(as.matrix(perf_amputed_test@metrics$cm$table[1:2,1:2]))
  measures_full_test<-confusionMatrix(cm_amputed_mnar_test,positive = "1")
  perf_measure_amputed_mnar_test[i,1:4]<-matrix(c(measures_full_test$overall[1],measures_full_test$byClass[1:2],
                                               perf_amputed_test@metrics$AUC),ncol=4)
  
  ############################
  ###########Models and Performance for Imputed data
  model_imputed_mar<- h2o.grid("deeplearning",
                               x=2:(numnor+1), y=1,
                               nfolds=folds, 
                               hyper_params = hyper_param,
                               training_frame =imputed_mar_data_train_h2o,
                               activation="Tanh")
  
  
  #**** Get the grid results, sorted by validation AUC
  model_imputed_mar_grid <- h2o.getGrid(grid_id = model_imputed_mar@grid_id,
                                        sort_by = "auc",
                                        decreasing = TRUE)
  
  #**** Grab the top Deep Learning model, chosen by validation AUC
  best_imputed_mar <- h2o.getModel(model_imputed_mar_grid@model_ids[[1]])
  
  #**** Now let's evaluate the model performance 
  perf_imputed_train<-h2o.performance(best_imputed_mar)
  
  cm_imputed_mar_train<-as.table(as.matrix(perf_imputed_train@metrics$cm$table[1:2,1:2]))
  measures_full_train<-confusionMatrix(cm_imputed_mar_train,positive = "1")
  perf_measure_imputed_mar_train[i,1:4]<-matrix(c(measures_full_train$overall[1],measures_full_train$byClass[1:2],
                                               perf_imputed_train@metrics$AUC),ncol=4)
  
  perf_imputed_test<-h2o.performance(best_imputed_mar,newdata = imputed_mar_data_test_h2o) #***test performance
  
  cm_imputed_mar_test<-as.table(as.matrix(perf_imputed_test@metrics$cm$table[1:2,1:2]))
  measures_full_test<-confusionMatrix(cm_imputed_mar_test,positive = "1")
  perf_measure_imputed_mar_test[i,1:4]<-matrix(c(measures_full_test$overall[1],measures_full_test$byClass[1:2],
                                              perf_imputed_test@metrics$AUC),ncol=4)
  
  model_imputed_mcar<-h2o.grid("deeplearning",
                               x=2:(numnor+1), y=1,
                               nfolds=folds, 
                               hyper_params = hyper_param,
                               training_frame =imputed_mcar_data_train_h2o,
                               activation="Tanh")
  
  #**** Get the grid results, sorted by validation AUC
  model_imputed_mcar_grid <- h2o.getGrid(grid_id = model_imputed_mcar@grid_id,
                                         sort_by = "auc",
                                         decreasing = TRUE)
  
  #**** Grab the top Deep Learning model, chosen by validation AUC
  best_imputed_mcar <- h2o.getModel(model_imputed_mcar_grid@model_ids[[1]])
  
  #**** Now let's evaluate the model performance 
  perf_imputed_train<-h2o.performance(best_imputed_mcar)
  
  cm_imputed_mcar_train<-as.table(as.matrix(perf_imputed_train@metrics$cm$table[1:2,1:2]))
  measures_full_train<-confusionMatrix(cm_imputed_mcar_train,positive = "1")
  perf_measure_imputed_mcar_train[i,1:4]<-matrix(c(measures_full_train$overall[1],measures_full_train$byClass[1:2],
                                                perf_imputed_train@metrics$AUC),ncol=4)
  
  perf_imputed_test<-h2o.performance(best_imputed_mcar,newdata = imputed_mcar_data_test_h2o) #***test performance
  
  cm_imputed_mcar_test<-as.table(as.matrix(perf_imputed_test@metrics$cm$table[1:2,1:2]))
  measures_full_test<-confusionMatrix(cm_imputed_mcar_test,positive = "1")
  perf_measure_imputed_mcar_test[i,1:4]<-matrix(c(measures_full_test$overall[1],measures_full_test$byClass[1:2],
                                               perf_imputed_test@metrics$AUC),ncol=4)
  
  model_imputed_mnar<- h2o.grid("deeplearning",
                                x=2:(numnor+1), y=1,
                                nfolds=folds, 
                                hyper_params = hyper_param,
                                training_frame =imputed_mnar_data_train_h2o,
                                activation="Tanh")
  
  #**** Get the grid results, sorted by validation AUC
  model_imputed_mnar_grid <- h2o.getGrid(grid_id = model_imputed_mnar@grid_id,
                                         sort_by = "auc",
                                         decreasing = TRUE)
  
  #**** Grab the top Deep Learning model, chosen by validation AUC
  best_imputed_mnar <- h2o.getModel(model_imputed_mnar_grid@model_ids[[1]])
  
  #**** Now let's evaluate the model performance 
  perf_imputed_train<-h2o.performance(best_imputed_mnar)
  
  cm_imputed_mnar_train<-as.table(as.matrix(perf_imputed_train@metrics$cm$table[1:2,1:2]))
  measures_full_train<-confusionMatrix(cm_imputed_mnar_train,positive = "1")
  perf_measure_imputed_mnar_train[i,1:4]<-matrix(c(measures_full_train$overall[1],measures_full_train$byClass[1:2],
                                                perf_imputed_train@metrics$AUC),ncol=4)
  
  perf_imputed_test<-h2o.performance(best_imputed_mnar,newdata = imputed_mnar_data_test_h2o) #***test performance
  
  cm_imputed_mnar_test<-as.table(as.matrix(perf_imputed_test@metrics$cm$table[1:2,1:2]))
  measures_full_test<-confusionMatrix(cm_imputed_mnar_test,positive = "1")
  perf_measure_imputed_mnar_test[i,1:4]<-matrix(c(measures_full_test$overall[1],measures_full_test$byClass[1:2],
                                               perf_imputed_test@metrics$AUC),ncol=4)
  h2o.removeAll()
  
}

s106<- data.frame(cbind(perf_measure_train,perf_measure_test,perf_measure_amputed_mcar_train,
                        perf_measure_amputed_mcar_test,perf_measure_amputed_mar_train,
                        perf_measure_amputed_mar_test,perf_measure_amputed_mnar_train,
                        perf_measure_amputed_mnar_test,perf_measure_imputed_mcar_train,
                        perf_measure_imputed_mcar_test,perf_measure_imputed_mar_train,
                        perf_measure_imputed_mar_test,perf_measure_imputed_mnar_train,
                        perf_measure_imputed_mnar_test))
colnames(s106)<-c("full_train_acc","full_train_sens","full_train_spef","full_train_AUC",
                       "full_test_acc","full_test_sens","full_test_spef","full_test_AUC",
                       "amp_mcar_train_acc","amp_mcar_train_sens","amp_mcar_train_spef","amp_mcar_train_AUC",
                       "amp_mcar_test_acc","amp_mcar_test_sens","amp_mcar_test_spef","amp_mcar_test_AUC",
                       "amp_mar_train_acc","amp_mar_train_sens","amp_mar_train_spef","amp_mar_train_AUC",
                       "amp_mar_test_acc","amp_mar_test_sens","amp_mar_test_spef","amp_mar_test_AUC",
                       "amp_mnar_train_acc","amp_mnar_train_sens","amp_mnar_train_spef","amp_mnar_train_AUC",
                       "amp_mnar_test_acc","amp_mnar_test_sens","amp_mnar_test_spef","amp_mnar_test_AUC",
                       "imp_mcar_train_acc","imp_mcar_train_sens","imp_mcar_train_spef","imp_mcar_train_AUC",
                       "imp_mcar_test_acc","imp_mcar_test_sens","imp_mcar_test_spef","imp_mcar_test_AUC",
                       "imp_mar_train_acc","imp_mar_train_sens","imp_mar_train_spef","imp_mar_train_AUC",
                       "imp_mar_test_acc","imp_mar_test_sens","imp_mar_test_spef","imp_mar_test_AUC",
                       "imp_mnar_train_acc","imp_mnar_train_sens","imp_mnar_train_spef","imp_mnar_train_AUC",
                       "imp_mnar_test_acc","imp_mnar_test_sens","imp_mnar_test_spef","imp_mnar_test_AUC")

